In [8]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras import layers,models
import os

In [45]:
training_path = os.path.join("monkey/training/training")
validation_path = os.path.join("monkey/validation/validation")

In [47]:
path = os.listdir(training_path)

In [13]:
training_generator = ImageDataGenerator(
    horizontal_flip = True,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    rescale = 1./255.,
    rotation_range = 20,
    )
train_data = training_generator.flow_from_directory(
    training_path, batch_size = 32, target_size=(150, 150), class_mode = "categorical", shuffle = True)

Found 1098 images belonging to 10 classes.


In [14]:
val_gen = ImageDataGenerator(
    rescale = 1./255.
)
val_data = val_gen.flow_from_directory(validation_path, batch_size = 1, target_size = (150,150),class_mode = "categorical", shuffle = True)

Found 272 images belonging to 10 classes.


In [15]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image

In [16]:
model = InceptionV3(weights='imagenet', include_top=False, input_shape = (150,150,3))
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 74, 74, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 74, 74, 32)   0           batch_normalization_282[0][0]    
_______________________________________________________________________________________

In [21]:
model.trainable = False
add_model = models.Sequential()
add_model.add(model)
add_model.add(layers.GlobalAveragePooling2D())
add_model.add(layers.Dropout(0.5))
add_model.add(layers.Dense(10, 
                    activation='softmax'))

In [38]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.01)
add_model.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=['accuracy'])
add_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________


In [41]:
history = add_model.fit(train_data, 
                              epochs=6)

Epoch 1/2
35/35 [==============================] - 48s 1s/step - loss: 0.6678 - accuracy: 0.8297
Epoch 2/2
35/35 [==============================] - 47s 1s/step - loss: 0.5807 - accuracy: 0.8506


In [42]:
add_model.evaluate(val_data)

272/272 [==============================] - 31s 59ms/step - loss: 0.2390 - accuracy: 0.9301


[0.23896096646785736, 0.9301470518112183]

In [89]:
import random
y_test = np.zeros((500,10))
x_test = []
image_names = []

class_dict = {"n0":0,"n1":1,"n2":2,"n3":3,"n4":4,"n5":5,"n6":6,"n7":7,"n8":8,"n9":9}
for clas in path:
    for i in range(50):
        images = os.listdir(training_path+"/"+clas)
        random_name = random.choice(images)
        img = image.load_img(os.path.join(training_path+"/"+clas+"/"+random_name), target_size=(150, 150) )
        image_names.append(training_path+"/"+clas+"/"+random_name)
        x_test.append(np.array(img))
        y_test[(class_dict[clas]*50)+i][class_dict[clas]] = 1
    
    
x_test = np.asarray(x_test)

In [81]:
x_test = x_test/255.

In [82]:
res = add_model.predict_classes(x_test)

In [83]:
res

array([0, 0, 0, 0, 0, 0, 9, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3,
       3, 3, 4, 4, 8, 4, 4, 4, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6,

In [72]:
actual = np.argmax(y_test, axis = -1)

In [86]:
pred= {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}
for i,j in zip(actual,res):
    if i == j:
        pred[i] = pred[i]+1

In [88]:
pred

{0: 47, 1: 50, 2: 50, 3: 47, 4: 48, 5: 49, 6: 50, 7: 44, 8: 50, 9: 47}

In [90]:
reverse_dict = {0:"mantled_howler",1:"patas_monkey",2:"bald_uakari",3:"japanese_macaque",4:"pygmy_marmoset",5:"white_headed_capuchin",6:"silvery_marmoset",7:"common_squirrel_monkey",8:"black_headed_night_monkey",9:"nilgiri_langur"}

In [94]:
print("Image Name              \t    Predicted Monkey Name    \t Actual Monkey Name")
for i,j in enumerate(image_names):
    print(j,"\t",reverse_dict[res[i]],"\t",reverse_dict[actual[i]])

Image Name              	    Predicted Monkey Name    	 Actual Monkey Name
monkey/training/training/n0/n0137.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0019.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0119.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0163.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0176.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0120.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0044.jpg 	 nilgiri_langur 	 mantled_howler
monkey/training/training/n0/n0167.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0043.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0112.jpg 	 white_headed_capuchin 	 mantled_howler
monkey/training/training/n0/n0037.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0040.jpg 	 mantled_howler 	 mantled_howler
monkey/training/training/n0/n0169.jpg 	 mantled_howler

monkey/training/training/n5/n5126.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5137.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5056.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5134.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5142.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5064.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5149.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5047.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5060.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5034.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5156.jpg 	 white_headed_capuchin 	 white_headed_capuchin
monkey/training/training/n5/n5018.jpg 	 white_headed_c